# Introduction

This notebook shows how to work with the weaviate streamlit connector.

# Imports

In [1]:
from st_weaviate_connection import WeaviateConnection
import streamlit as st
import os

/workspaces/st-weaviate-connection/.venv/lib/python3.11/site-packages/streamlit/elements/image.py:22: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


# Usage

## Making A Connection

In [2]:
conn = st.experimental_connection(
    "weaviate",
    type=WeaviateConnection,
    url=os.getenv("WEAVIATE_URL"),
    api_key=os.getenv("WEAVIATE_API_KEY"),
)

2023-07-27 05:56:08.056 
  command:

    streamlit run /workspaces/st-weaviate-connection/.venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


## Uploading Data

Make up some data:

In [3]:
pokemons = [
    {
        "name": "Pikachu",
        "type": "Electric",
        "hp": 35,
        "attack": 55,
        "defense": 40,
        "speed": 90,
    },
    {
        "name": "Charmander",
        "type": "Fire",
        "hp": 39,
        "attack": 52,
        "defense": 43,
        "speed": 65,
    },
    {
        "name": "Squirtle",
        "type": "Water",
        "hp": 44,
        "attack": 48,
        "defense": 65,
        "speed": 43,
    },
    {
        "name": "Bulbasaur",
        "type": "Grass",
        "hp": 45,
        "attack": 49,
        "defense": 49,
        "speed": 45,
    },
]

Use the client to upload the data:

In [4]:
with conn.client().batch as batch:
    for pokemon in pokemons:
        batch.add_data_object(data_object=pokemon, class_name="Pokemon")

## Querying Data

We can query data using the `query` method which accepts a query straing in graphql:

In [5]:
gql = """
{
    Get {
        Pokemon(limit: 3, bm25: {query: "electric"}) {
        name
        attack
        _additional {
            score
        }
        }
    }
}
"""

df = conn.query(gql)
df

2023-07-27 05:56:08.742 No runtime found, using MemoryCacheStorageManager
2023-07-27 05:56:08.746 No runtime found, using MemoryCacheStorageManager


,attack,name,_additional.score
0,55,Pikachu,0.54726034


We can also using the weaviate client's builder pattern:

In [6]:
results = conn.client().query.get("Pokemon", ["name", "attack"])\
    .with_limit(3)\
    .with_additional("score")\
    .with_bm25(query="electric")\
    .do()

results

{'data': {'Get': {'Pokemon': [{'_additional': {'score': '0.54726034'},
     'attack': 55,
     'name': 'Pikachu'}]}}}

## Using Secrets

The following are valid values you can specify in your `secrets.toml` file when using this connection:

| Config | Description |
| --- | --- |
| WEAVIATE_URL | The url of the weavite instance you want to connect to |
| WEAVIATE_API_KEY | The corresponding api key of the weaviate instance you want to connect to (if applicable) |

For more details, refer to:

* [How to use secrets management](https://docs.streamlit.io/library/advanced-features/secrets-management#how-to-use-secrets-management)
* [st.experimental_connection](https://docs.streamlit.io/library/api-reference/connections/st.experimental_connection#stexperimental_connection)